In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [5]:
from expiries import dict_expiries

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [7]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{variation}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [8]:
bnf_1min = pd.read_csv("../data/sensex_min.csv")

In [9]:
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
# bnf_1min.tail()

In [10]:
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
# bnf_1min.tail()

In [11]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
]

In [12]:
from datetime import date
from bisect import bisect_left

def get_next_expiry(input_date, index_symbol):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_left(expiry_dates, input_date)
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [13]:
index_ = 'sensex'

if index_ == 'sensex':
    LOT_SIZE_ = 20
    STRIKE_SPREAD_ = 100
    INDEX_LEVERAGE_ = 8
    PORTFOLIO_ = 1_00_00_000

In [67]:
def fetch_spot_open(current_date, df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    spot_df = df[(df['datetime'].dt.date == current_date) & (df['datetime'].dt.time >= dt.time(9, 15))]
    print(spot_df.head().to_string())
    if spot_df is not None and not spot_df.empty:
        return int(round(spot_df.iloc[0]['o'] / STRIKE_SPREAD_) * STRIKE_SPREAD_)  # open price at 9:15
    return None

def generate_strike_list(spot_open):
    spot_atm = round(spot_open / STRIKE_SPREAD_) * STRIKE_SPREAD_  # Nearest 100
    return list(range(spot_atm - (10 * STRIKE_SPREAD_), spot_atm + (10 * STRIKE_SPREAD_) + 1, STRIKE_SPREAD_))

In [68]:
# Strategy Parameters
START_DATE = dt.date(2025, 1, 1)
END_DATE = dt.date(2025, 1, 3)
TARGET = 10 
INDEX_ = 'sensex'
RISK_FREE_RETURN_ = 0.06
results = []

In [118]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import math
import datetime

#     # S = Spot price
#     # K = Strike price
#     # T = Time to expiry (1 day)
#     # r = Risk-free rate (6% annual)

def calculate_time_to_expiry(df, expiry):
    import pandas as pd

    # Convert expiry to pandas Timestamp with time (add time 15:30:00 if expiry is a date)
    if isinstance(expiry, pd.Timestamp):
        expiry_dt = expiry
    elif isinstance(expiry, (str, pd.Timestamp)):
        expiry_dt = pd.to_datetime(expiry)
    elif isinstance(expiry, datetime.date) and not isinstance(expiry, datetime.datetime):
        # Add time 15:30:00 or whichever time your expiry corresponds to
        expiry_dt = pd.Timestamp(datetime.datetime.combine(expiry, datetime.time(15,30)))
    else:
        expiry_dt = pd.Timestamp(expiry)

    # Calculate time difference
    df['T'] = (expiry_dt - df['datetime']).dt.total_seconds() / (365 * 24 * 3600)

    # Clip or NaN for negative/zero
    df.loc[df['T'] < 1e-6, 'T'] = np.nan

    return df

# Black-Scholes Pricing Formula
def bs_price(option_type, S, K, T, r, sigma):
    if T <= 0:
        return 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'C':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'P':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'C' or 'P'")

# IV solver using Brent's method
def compute_iv(row, spot_price, r, strike, option_type, expiry):
    try:
        K = row['strike']
        price = row['c']  # You can change to 'c' or 'mid' if preferred
        T = max((expiry - row['datetime'].date()).days + (1/365), 1e-6) / 365  # time in years

        if price < 0.1 or T <= 0:  # Avoid noise
            return np.nan

        iv = brentq(
            lambda sigma: bs_price(option_type, spot_price, K, T, r, sigma) - price,
            1e-5, 5.0
        )
        return iv
    except Exception:
        return np.nan

# Main function to add 'IV' column
def add_iv_column(df, r, strike, opt_type, expiry):
    r = 0.07
    df = df.copy()
    df = calculate_time_to_expiry(df, expiry)
    df['IV'] = df.apply(lambda row: compute_iv(row, row['spot_price'], r, strike, opt_type, expiry), axis=1)
    return df


In [119]:
import pandas as pd

def add_spot_price_column(option_df: pd.DataFrame, spot_df: pd.DataFrame, start_date, end_date) -> pd.DataFrame:
    """
    Adds a 'spot_price' column to option_df by merging 'c' from spot_df (close price)
    based on datetime alignment, using a filtered spot_df between start_date and end_date.

    Parameters:
        option_df (pd.DataFrame): Option data with 'datetime' column.
        spot_df (pd.DataFrame): Spot data with 'datetime' and OHLC columns.
        start_date (str or pd.Timestamp): Start datetime for slicing spot_df.
        end_date (str or pd.Timestamp): End datetime for slicing spot_df.

    Returns:
        pd.DataFrame: option_df with 'spot_price' column added.
    """
    # Copy inputs
    option_df = option_df.copy()
    spot_df = spot_df.copy()

    # Ensure datetime conversion
    option_df['datetime'] = pd.to_datetime(option_df['datetime'])
    spot_df['datetime'] = pd.to_datetime(spot_df['datetime'])

    # Filter spot_df to reduce size
    filtered_spot_df = spot_df[
        (spot_df['datetime'] >= pd.to_datetime(start_date)) &
        (spot_df['datetime'] <= pd.to_datetime(end_date))
    ].copy()

    # Sort both DataFrames
    option_df.sort_values('datetime', inplace=True)
    filtered_spot_df.sort_values('datetime', inplace=True)

    # Select only 'datetime' and 'c' column from spot_df, rename to 'spot_price'
    spot_price_df = filtered_spot_df[['datetime', 'c']].rename(columns={'c': 'spot_price'})

    # Ensure datetime columns are of the same dtype
    option_df['datetime'] = option_df['datetime'].astype('datetime64[ns]')
    spot_price_df['datetime'] = spot_price_df['datetime'].astype('datetime64[ns]')

    print(option_df.columns)
    print(spot_price_df.columns)

    print(option_df['datetime'].dtype)       # should be datetime64[ns]
    print(spot_price_df['datetime'].dtype)   # should also be datetime64[ns]
    
    # Merge using nearest earlier timestamp
    merged_df = pd.merge_asof(
        option_df.sort_values('datetime'),
        spot_price_df.sort_values('datetime'),
        on='datetime',
        direction='backward'
    )

    # Forward fill any remaining NaNs in spot_price
    merged_df['spot_price'] = merged_df['spot_price'].ffill()

    return merged_df


In [120]:
async def execute_strategy(df):
    current_date = START_DATE
    while current_date <= END_DATE:
        expiry = get_next_expiry(current_date, INDEX_)
        if expiry != current_date:
            print(f'Skipping : {current_date}')
            current_date += dt.timedelta(days=1)
            continue  # Skip non-0DTE days
        print(current_date)
        spot_atm = fetch_spot_open(current_date, df) + 500
        print(spot_atm)
        SENSEX_STRIKES = generate_strike_list(spot_atm)
        
        # for strike in SENSEX_STRIKES:
            # print(strike)
        start_dt = dt.datetime.combine(current_date, dt.time(9, 15))
        end_dt = dt.datetime.combine(current_date, dt.time(15, 30))
        ce_df = await fetch_data(
            index=INDEX_,
            expiry=expiry,
            strike=spot_atm,
            asset_class='C',
            start_date=current_date - dt.timedelta(days=4),
            start_time=dt.time(9, 15),
            end_date=current_date,
            end_time=dt.time(15, 30),
        )
        if ce_df is not None and not isinstance(ce_df, str):
            ce_df = ce_df.to_pandas()
            print(ce_df.tail().to_string())
            # Find if the option opens below 5 points
            ce_df = add_spot_price_column(ce_df, df, current_date - dt.timedelta(days=4), current_date)
            ce_df = add_iv_column(ce_df, RISK_FREE_RETURN_, spot_atm, 'C', expiry)
            print(ce_df.to_string())
            # for i in range(0, len(ce_df)):
            #     low_price = ce_df.iloc[i]['l']
                
            #     if low_price <= 5:
            #         # Look for target or EOD exit
            #         entry_time = ce_df['datetime']
            #         entry_price = 
            #         exit_price = None
            #         exit_time = None
            #         for i, row in df.iterrows():
            #             if row['h'] >= entry_price * TARGET:
            #                 exit_price = entry_price * TARGET
            #                 exit_time = row['datetime']
            #                 break
            #         else:
            #             exit_price = df.iloc[-1]['c']
            #             exit_time = df.iloc[-1]['datetime']
    
            #         results.append({
            #             "date": current_date,
            #             "strike": strike,
            #             "entry_price": entry_price,
            #             "exit_price": exit_price,
            #             "exit_time": exit_time,
            #             "pnl": exit_price - entry_price
            #         })
            #         break  # Only take first valid trade for the day

        current_date += dt.timedelta(days=1)
    return
    # Convert to DataFrame for analysis
    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df

In [121]:
await execute_strategy(bnf_1min)

Skipping : 2025-01-01
Skipping : 2025-01-02
2025-01-03
         index            datetime          o          h          l          c  v
626083  sensex 2025-01-03 09:15:00 79903.7500 79903.7500 79762.6700 79815.1200  0
626084  sensex 2025-01-03 09:16:00 79813.0700 79846.5700 79805.7800 79833.8200  0
626085  sensex 2025-01-03 09:17:00 79832.5400 79841.6800 79800.2300 79841.5000  0
626086  sensex 2025-01-03 09:18:00 79840.1800 79885.9300 79832.6400 79883.0500  0
626087  sensex 2025-01-03 09:19:00 79885.0800 79906.3600 79856.1400 79858.8000  0
80400
       index     expiry  strike asset_class            datetime      o      h      l      c       v      oi
1870  sensex 2025-01-03   80400           C 2025-01-03 15:25:00 0.0500 0.1000 0.0500 0.0500   14010  442360
1871  sensex 2025-01-03   80400           C 2025-01-03 15:26:00 0.1000 0.1000 0.0500 0.0500  122770  442360
1872  sensex 2025-01-03   80400           C 2025-01-03 15:27:00 0.0500 0.0500 0.0500 0.0500   53590  450590
1873  sensex 20